In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('nytaxi2022.csv')
df.head()

/var/folders/27/1nzvqhpd5f7bc_n8rgyv1v4h0000gn/T/ipykernel_18279/1526527318.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('nytaxi2022.csv')


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,01/01/2022 12:35:40 AM,01/01/2022 12:53:29 AM,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,01/01/2022 12:33:43 AM,01/01/2022 12:42:07 AM,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,01/01/2022 12:53:21 AM,01/01/2022 01:02:19 AM,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,01/01/2022 12:25:21 AM,01/01/2022 12:35:23 AM,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,01/01/2022 12:36:48 AM,01/01/2022 01:14:20 AM,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [4]:
df.shape

(39656098, 19)

In [10]:
import pandas as pd

# convert "" or whitespace-only cells to NA so pandas counts them as missing
df = df.replace(r"^\s*$", pd.NA, regex=True)

In [11]:
# number of missing row within a columns
df.isna().sum()            # per-column counts

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1368303
trip_distance                  0
RatecodeID               1368303
store_and_fwd_flag       1368303
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1368303
airport_fee              1368303
dtype: int64

In [12]:
N = len(df)
cols = ["passenger_count","RatecodeID","store_and_fwd_flag","congestion_surcharge","airport_fee"]

miss_tbl = (
    df[cols].isna().sum()
      .to_frame("missing")
      .assign(pct=lambda t: (t["missing"]/N*100).round(2))
)
miss_tbl  # quick table

# Are the *same* rows missing all five?
co_missing = df[cols].isna().all(axis=1)
co_missing.sum(), (co_missing.mean()*100).round(2)  # count & %

(np.int64(1368303), np.float64(3.45))

Output confirms that the same 1,368,303 (~3.45%) are missing values in these columns: passenger_count, RatecodeID, store_and_fwd_flag, congestion_surcharge, airport_fee

In [14]:
TARGET = "total_amount"

allowed = ["tpep_pickup_datetime","tpep_dropoff_datetime",
    "passenger_count","trip_distance","RatecodeID",
    "PULocationID","DOLocationID","payment_type","extra"]

leakage = ["fare_amount","mta_tax","tip_amount","tolls_amount",
    "improvement_surcharge","congestion_surcharge","airport_fee", TARGET]

present = [c for c in df.columns if c in allowed]
leak_present = [c for c in df.columns if c in leakage and c != TARGET]

# optional: remove near-constant and very-missing among the allowed set
missing = df[present].isna().mean().sort_values(ascending=False)
const_like = [c for c in present if df[c].nunique(dropna=True) <= 1]
very_missing = [c for c in present if missing[c] > 0.40] #40% threshold; tweak

keep = [c for c in present if c not in const_like and c not in very_missing]

print("Keep: ", keep)
print("DROP (leakage): ", leak_present)
print("DROP (constant): ", const_like)
print("DROP (very missing): ", very_missing)

Keep:  ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID', 'payment_type', 'extra']
DROP (leakage):  ['fare_amount', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee']
DROP (constant):  []
DROP (very missing):  []


Engineer the time features column

In [15]:
import numpy as np
m = df[keep + [TARGET]].copy()
for c in ["tpep_pickup_datetime","tpep_dropoff_datetime"]:
    m[c] = pd.to_datetime(m[c], errors="coerce", utc=True)

m["trip_duration_min"] = (m["tpep_dropoff_datetime"] - m["tpep_pickup_datetime"]).dt.total_seconds()/60
m["pickup_hour"] = m["tpep_pickup_datetime"].dt.hour.astype("Int8")
m["pickup_dow"]  = m["tpep_pickup_datetime"].dt.dayofweek.astype("Int8")


/var/folders/27/1nzvqhpd5f7bc_n8rgyv1v4h0000gn/T/ipykernel_18279/3375058208.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  m[c] = pd.to_datetime(m[c], errors="coerce", utc=True)
/var/folders/27/1nzvqhpd5f7bc_n8rgyv1v4h0000gn/T/ipykernel_18279/3375058208.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  m[c] = pd.to_datetime(m[c], errors="coerce", utc=True)


In [16]:
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_dt

# required columns present?
req = ["tpep_pickup_datetime","tpep_dropoff_datetime",
       "trip_duration_min","pickup_hour","pickup_dow"]
print("all present:", set(req).issubset(m.columns))

# dtypes look right?
print("pickup dtype:", m["tpep_pickup_datetime"].dtype, "| dropoff dtype:", m["tpep_dropoff_datetime"].dtype)
print("duration dtype:", m["trip_duration_min"].dtype, "| hour dtype:", m["pickup_hour"].dtype)

# NAs
m[req].isna().sum()


all present: True
pickup dtype: datetime64[ns, UTC] | dropoff dtype: datetime64[ns, UTC]
duration dtype: float64 | hour dtype: Int8


tpep_pickup_datetime     0
tpep_dropoff_datetime    0
trip_duration_min        0
pickup_hour              0
pickup_dow               0
dtype: int64

Time-based split (avoid leakage)

In [17]:
TARGET = "total_amount"
NUM = ["passenger_count","trip_distance","extra","trip_duration_min","pickup_hour","pickup_dow"]
CAT = ["RatecodeID","PULocationID","DOLocationID","payment_type"]

ms = m.sort_values("tpep_pickup_datetime")
cut = int(len(ms)*0.70)
train, test = ms.iloc[:cut], ms.iloc[cut:]

Xtr, ytr = train[NUM + CAT], train[TARGET].astype("float32")
Xte, yte = test [NUM + CAT], test [TARGET].astype("float32")

len(train), len(test)


(27759268, 11896830)

* Splitting by time to match how the model will be used in real lift: trained on the past, predicts the future.

* A random split can quietly inflate scores for time-dependent data like taxi trips

Why time-based split?
* Realistic evaluation: Testing on later periods simulates that
* Avoid look-ahead leakage: Global steps (impute medians, scaling, ont-hot categories) must be learned on train only. If future rows is mix into training data, future information might be leak
* Handles temporal correlation: Trips close in time share weather, traffic, events, policy changes. Random splits sprinkle near-identical conditions across train/test -> over-optimistic metrics
* Detects drift: Fare/behavior shift by month/season. A chronological split shows how well the moel generalizes to a new regime

In [21]:
import sklearn, sys, inspect
print("sklearn:", sklearn.__version__)
print("python:", sys.executable)
inspect.signature(__import__("sklearn.metrics").metrics.mean_squared_error)


sklearn: 1.7.2
python: /Users/chrisnjw/.pyenv/versions/myproj-311/bin/python


<Signature (y_true, y_pred, *, sample_weight=None, multioutput='uniform_average')>

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(yte, pred)

try:
    rmse = mean_squared_error(yte, pred, squared=False)  # new API
except TypeError:
    rmse = np.sqrt(mean_squared_error(yte, pred))        # fallback for old sklearn

r2  = r2_score(yte, pred)
(mae, rmse, r2)

(6.399168288871747, np.float64(55.5506942344371), 0.07257096419322462)

In [23]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

baseline = np.full_like(yte, fill_value=float(ytr.mean()))
b_mae = mean_absolute_error(yte, baseline)
try:
    b_rmse = mean_squared_error(yte, baseline, squared=False)
except TypeError:
    b_rmse = np.sqrt(mean_squared_error(yte, baseline))

mae, rmse, r2, b_mae, b_rmse

(6.399168288871747,
 np.float64(55.5506942344371),
 0.07257096419322462,
 11.60715389251709,
 np.float64(57.711281748161035))

Baseline (Ridge) with proper preprocessing in a pipleline

In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

TARGET = "total_amount"
NUM = ["passenger_count","trip_distance","extra","trip_duration_min","pickup_hour","pickup_dow"]
CAT = ["RatecodeID","PULocationID","DOLocationID","payment_type"]

Xtr, ytr = train[NUM+CAT], train[TARGET].astype("float32")
Xte, yte = test [NUM+CAT], test [TARGET].astype("float32")

pre_tree = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), NUM),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("oh",  OneHotEncoder(handle_unknown="ignore", min_frequency=5))  # collapse rare cats
    ]), CAT),
])

rf = Pipeline([
    ("pre", pre_tree),
    ("est", RandomForestRegressor(
        n_estimators=300,
        max_depth=20,            # cap depth for speed/overfit control
        min_samples_leaf=5,
        n_jobs=-1,
        random_state=42
    ))
])

# warm up on a sample; then remove these two lines for full fit
idx = Xtr.sample(n=1_000_000, random_state=42).index
rf.fit(Xtr.loc[idx], ytr.loc[idx])

# quick eval on a slice; then remove head(...) for full eval
pred = rf.predict(Xte.head(500_000))

# metrics (robust to sklearn version)
mae = mean_absolute_error(yte.head(500_000), pred)
try:
    rmse = mean_squared_error(yte.head(500_000), pred, squared=False)
except TypeError:
    rmse = np.sqrt(mean_squared_error(yte.head(500_000), pred))
r2 = r2_score(yte.head(500_000), pred)
(mae, rmse, r2)


(1.8779829572318527, np.float64(6.585973441997223), 0.8788138401564854)

y = dollar fare I am predicting
X = numeric + categorical predictors based on my time-based split

* Train 300 trees, each grown (up to depth 20) on the 1M training rows after OHE 

MAE = 1.88 dollars
RMSE = 6.59
R^2 = 0.879